In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

## Creating Batch according to the set Batch Size

In [2]:
def make_batch():
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]

        input_batch.append(input)
        target_batch.append(target)

    return input_batch, target_batch

## NNLM Model Structure

In [3]:
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__()
        self.C = nn.Embedding(n_class, m)
        self.H = nn.Linear(n_step*m, n_hidden, bias=False)
        self.d = nn.Parameter(torch.ones(n_hidden))
        self.U = nn.Linear(n_hidden, n_class, bias=False)
        self.W = nn.Linear(n_step*m, n_class, bias=False)
        self.b = nn.Parameter(torch.ones(n_class))

    def forward(self, X):
        X = self.C(X)
        X = X.view(-1, n_step * m)
        tanh = torch.tanh(self.d + self.H(X))
        output = self.b + self.W(X) + self.U(tanh)
        return output

## Prediction

In [4]:
sentences = ['i like dog', 'i love coffee', 'i hate milk', 'i see cat', 'i inject caffeine']

word_list = " ".join(sentences[:]).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)

n_step = 2
n_hidden = 2
m = 2

model = NNLM()
print(NNLM())
print('-' * 50)
print(f"Word Dictionary : {word_dict}")
print('-' * 50)
print(f"Number Dictionary : {number_dict}")
print('-' * 50)
print(f"Number of Class : {n_class}")
print('-' * 50)

model = NNLM()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

input_batch, target_batch = make_batch()
input_batch = torch.LongTensor(input_batch)
target_batch = torch.LongTensor(target_batch)



for epoch in range(5000):
    optimizer.zero_grad()
    output = model(input_batch)

    loss = criterion(output, target_batch)
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

predict = model(input_batch).data.max(1, keepdim=True)[1]

print('-' * 50)
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

NNLM(
  (C): Embedding(11, 2)
  (H): Linear(in_features=4, out_features=2, bias=False)
  (U): Linear(in_features=2, out_features=11, bias=False)
  (W): Linear(in_features=4, out_features=11, bias=False)
)
--------------------------------------------------
Word Dictionary : {'inject': 0, 'hate': 1, 'like': 2, 'coffee': 3, 'love': 4, 'cat': 5, 'caffeine': 6, 'i': 7, 'milk': 8, 'dog': 9, 'see': 10}
--------------------------------------------------
Number Dictionary : {0: 'inject', 1: 'hate', 2: 'like', 3: 'coffee', 4: 'love', 5: 'cat', 6: 'caffeine', 7: 'i', 8: 'milk', 9: 'dog', 10: 'see'}
--------------------------------------------------
Number of Class : 11
--------------------------------------------------
Epoch: 1000 cost = 0.190615
Epoch: 2000 cost = 0.034957
Epoch: 3000 cost = 0.012635
Epoch: 4000 cost = 0.005818
Epoch: 5000 cost = 0.002975
--------------------------------------------------
[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['i', 'see'], ['i', 'inject']] -> ['dog', 'co